# Apply Ranking Recovery Algorithms

In [1]:
import pandas as pd
import seaborn as sns

from fairpair import *

In [2]:
# create a FairPairGraph with disadvantaged minority
G = FairPairGraph()
G.generate_groups(15, 5)
G.group_assign_scores(nodes=G.majority_nodes, distr=Distributions.normal_distr)
G.group_assign_scores(nodes=G.minority_nodes, distr=Distributions.normal_distr, loc=0.3, scale=0.2) # give a disadvantage to the minority

In [3]:
# run 100 iterations of ProbKnockout sampling
sampler = OversampleMinority(G, warn=False)
sampler.apply(iter=1000, k=1)
# apply davidScore for ranking recovery
ranker = RankRecovery(G)
ranking, other_nodes = ranker.apply(rank_using=davidScore) # by default, apply rankCentrality method
ranker._print_with_score(ranking) # sorted by rank score

,node,orig score,rank score
0,2,0.761136,23.676924
1,10,0.617786,23.092642
2,3,0.826102,19.043764
3,9,0.622732,17.253319
4,7,0.722633,17.137120
5,8,0.635050,14.690321
6,1,0.440170,8.873224
7,6,0.526740,7.992650
8,11,0.304061,7.814948
9,12,0.285122,4.476610


## Metrics

In [4]:
# score-based metrics
print("Majority NDCG (by score):", NDCG(G.majority, ranking)) # NDCG is only calculated as a "in-group" comparison because it is normalized
print("Minority NDCG (by score):", NDCG(G.minority, ranking))
print("Majority MSE (by score):", MSE(G.majority, ranking))
print("Minority MSE (by score):", MSE(G.minority, ranking))

Majority NDCG (by score): 0.9869772999535159
Minority NDCG (by score): 1.0
Majority MSE (by score): 0.052217780978470796
Minority MSE (by score): 0.11892053859031684


In [5]:
# rank-based metrics
print("Majority mean error (by rank):", rank_mean_error(G, ranking, G.majority))
print("Minority mean error (by rank):", rank_mean_error(G, ranking, G.minority))
print("Majority MSE (by rank):", rank_MSE(G, ranking, G.majority))
print("Minority MSE (by rank):", rank_MSE(G, ranking, G.minority))
print("Majority Spearman's rho:", spearmanr(G, ranking, G.majority))
print("Minority Spearman's rho:", spearmanr(G, ranking, G.minority))

Majority mean error (by rank): -0.4
Minority mean error (by rank): 0.8
Majority MSE (by rank): 1.6
Minority MSE (by rank): 3.2
Majority Spearman's rho: PearsonRResult(statistic=0.9646007243427176, pvalue=6.582433185270773e-06)
Minority Spearman's rho: PearsonRResult(statistic=0.9710607611177229, pvalue=0.005883967536981273)


In [6]:
# weighted Kendall tau
print("Majority weighted Kendall tau:", weighted_tau(G, ranking, G.majority))
print("Minority weighted Kendall tau:", weighted_tau(G, ranking, G.minority))

Majority weighted Kendall tau: 0.16972620141371658
Minority weighted Kendall tau: 0.34396459555591646


In [7]:
print("Overall Spearman r:", spearmanr(G, ranking))
print('--')
print("Overall weighted Kendall tau:", weighted_tau(G, ranking))

Overall Spearman r: PearsonRResult(statistic=0.9400599400599404, pvalue=1.930584686081638e-07)
--
Overall weighted Kendall tau: 0.1525231057647277
